<a href="https://colab.research.google.com/github/xiashenzhen/geemap/blob/main/landast_%E5%8D%95%E6%99%AF%E5%88%86%E7%B1%BB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

无监督分类

In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-jason2024')

In [ ]:
import geemap
geemap.ee_initialize()

In [ ]:
Map = geemap.Map()

point = ee.Geometry.Point([-88.0664, 41.9411])

image = (
    ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
    .filterBounds(point)
    .filterDate('2022-01-01', '2022-12-31')
    .sort('CLOUD_COVER')
    .first()
    .select('SR_B[1-7]')
)

region = image.geometry()
image = image.multiply(0.0000275).add(-0.2).set(image.toDictionary())
vis_params = {'min': 0, 'max': 0.3, 'bands': ['SR_B5', 'SR_B4', 'SR_B3']}

Map.centerObject(region, 8)
Map.addLayer(image, vis_params, "Landsat-9")
Map

In [ ]:
geemap.get_info(image)

In [ ]:
image.get('DATE_ACQUIRED').getInfo()

'2022-06-17'

In [ ]:
image.get('CLOUD_COVER').getInfo()

0.45

In [ ]:
training = image.sample(
    **{
        # "region": region,
        'scale': 30,
        'numPixels': 5000,
        'seed': 0,
        'geometries': True,  # Set this to False to ignore geometries
    }
)

Map.addLayer(training, {}, 'Training samples')
Map

In [ ]:
geemap.ee_to_df(training.limit(5))

,SR_B1,SR_B2,SR_B3,SR_B4,SR_B5,SR_B6,SR_B7
0,0.047885,0.055117,0.089602,0.103325,0.281882,0.354758,0.283423
1,0.055393,0.063725,0.097413,0.110530,0.390673,0.280562,0.170783
2,0.063532,0.074615,0.104645,0.137012,0.260790,0.404588,0.355995
3,0.019505,0.042962,0.092930,0.107285,0.286585,0.313700,0.223747
4,0.039552,0.052120,0.111878,0.095845,0.378765,0.262165,0.154475


In [ ]:
n_clusters = 5
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)

In [ ]:
result = image.cluster(clusterer)
Map.addLayer(result.randomVisualizer(), {}, 'clusters')
Map

In [ ]:
legend_dict = {
    'Open Water': '#466b9f',
    'Developed, High Intensity': '#ab0000',
    'Developed, Low Intensity': '#d99282',
    'Forest': '#1c5f2c',
    'Cropland': '#ab6c28'

}

palette = list(legend_dict.values())

Map.addLayer(
    result, {'min': 0, 'max': 4, 'palette': palette}, 'Labelled clusters'
)
Map.add_legend(title='Land Cover Type',legend_dict=legend_dict , position='bottomright')
Map

Map(bottom=6400.0, center=[41.705728515237524, -86.737060546875], controls=(WidgetControl(options=['position',…

In [ ]:

geemap.download_ee_image(image, filename='unsupervised.tif', region=region, scale=90)

unsupervised.tif: |          | 0.00/379M (raw) [  0.0%] in 00:00 (eta:     ?)

There is no STAC entry for: None


In [ ]:
! pip install geedim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 40.6 MB/s eta 0:00:00
